In [1]:
import sys
import traceback
import json # New import for reading JSON data from file

# Wrap imports in try/except to catch missing libraries immediately
try:
    print("Importing libraries...")
    from dash import Dash, dcc, html, dash_table
    import dash_leaflet as dl
    from dash.dependencies import Input, Output
    # The animal_shelter class is not needed
    # from animal_shelter import AnimalShelter 
    from PIL import Image, ImageDraw, ImageFont
    import io
    import base64
    print("Libraries imported successfully.")
except ImportError as e:
    print(f"\nCRITICAL ERROR: Missing library. {e}")
    sys.exit(1)

###########################
# Data Manipulation / Model
###########################

# --- START: Data Loading from File ---
DATA_FILE_NAME = 'sample_animal_data.txt'
mongo_data = []

try:
    print(f"Reading data from {DATA_FILE_NAME}...")
    with open(DATA_FILE_NAME, 'r') as f:
        # Read each line, parse it as a JSON object, and append to mongo_data list
        for line in f:
            if line.strip(): # Skip empty lines
                mongo_data.append(json.loads(line))
    print(f"Loaded {len(mongo_data)} records from {DATA_FILE_NAME}.")

except FileNotFoundError:
    print(f"\nCRITICAL ERROR: Data file '{DATA_FILE_NAME}' not found.")
    print("Please make sure 'sample_animal_data.txt' is in the same directory as the notebook.")
    # Use empty data to prevent immediate crash if file is missing
    mongo_data = []
except json.JSONDecodeError as e:
    print(f"\nCRITICAL ERROR: Failed to parse JSON data in the file: {e}")
    mongo_data = []
# --- END: Data Loading from File ---

# Helper function to get unique values from list of dicts
def get_unique_values(data_list, key):
    return sorted(list(set(item.get(key) for item in data_list if item.get(key) is not None)))

# Determine available columns (keys) from the first record
columns = list(mongo_data[0].keys()) if mongo_data else []

# Generate options for dropdowns
age_options = get_unique_values(mongo_data, 'age_upon_outcome')
breed_options = get_unique_values(mongo_data, 'breed')
type_options = get_unique_values(mongo_data, 'animal_type')

#########################
# Logo Creation
#########################
def create_animal_logo(text="AAC Animals"):
    width = 300
    height = 100
    bg_color = (200, 220, 255)
    text_color = (50, 50, 50)

    img = Image.new('RGB', (width, height), bg_color)
    d = ImageDraw.Draw(img)

    try:
        font = ImageFont.truetype("DejaVuSans.ttf", 24)
    except IOError:
        font = ImageFont.load_default()

    try:
        text_width, text_height = d.textsize(text, font=font)
    except AttributeError:
        # Fallback for newer Pillow versions that use textbbox
        left, top, right, bottom = d.textbbox((0, 0), text, font=font)
        text_width = right - left
        text_height = bottom - top

    text_x = (width - text_width) // 2
    text_y = (height - text_height) // 2

    d.text((text_x, text_y), text, fill=text_color, font=font)

    # Draw paws
    paw_color = (100, 100, 100)
    paw_size = 15
    paw_centers = [(35, 60), (265, 60)]

    for center_x, center_y in paw_centers:
        # Main pad
        d.ellipse((center_x - paw_size * 0.7, center_y - paw_size, center_x + paw_size * 0.7, center_y + paw_size), fill=paw_color)
        # Toes - simplified
        d.ellipse((center_x - paw_size * 0.9, center_y - paw_size * 1.5, center_x - paw_size * 0.3, center_y - paw_size * 0.9), fill=paw_color)
        d.ellipse((center_x - paw_size * 0.3, center_y - paw_size * 1.6, center_x + paw_size * 0.3, center_y - paw_size * 1.0), fill=paw_color)
        d.ellipse((center_x + paw_size * 0.3, center_y - paw_size * 1.5, center_x + paw_size * 0.9, center_y - paw_size * 0.9), fill=paw_color)

    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='PNG')
    encoded_image = base64.b64encode(img_byte_arr.getvalue()).decode('ascii')
    return f'data:image/png;base64,{encoded_image}'

logo_url = create_animal_logo()

#########################
# Dashboard Layout / View
#########################
print("Setting up Dash layout...")
app = Dash(__name__)

app.layout = html.Div([
    html.Center([
        html.Img(src=logo_url, height='80px'),
        html.B(html.H1('Austin Animal Center Outcomes - Data File'))
    ]),
    html.Hr(),
    html.Div([
        html.Label("Age Upon Outcome"),
        dcc.Dropdown(
            id='age-dropdown',
            options=[{'label': age, 'value': age} for age in age_options],
            placeholder="Select Age Upon Outcome"
        ),
        html.Label("Breed"),
        dcc.Dropdown(
            id='breed-dropdown',
            options=[{'label': breed, 'value': breed} for breed in breed_options],
            placeholder="Select Breed"
        ),
        html.Label("Animal Type"),
        dcc.Dropdown(
            id='type-dropdown',
            options=[{'label': animal_type, 'value': animal_type} for animal_type in type_options],
            placeholder="Select Animal Type"
        ),
        html.Br(),
        html.Div(id='filtered-results'),
        html.Label("Visual Representation of Animal Types"),
        dcc.Graph(id='animal-type-graph')
    ]),
    html.Hr(),
    html.P("Input specific search options in the first row of each category.", style={'fontSize': '15px', 'color': 'blue'}),
    dash_table.DataTable(
        id='datatable-id',
        # Handle case where columns might be empty if data load failed
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in columns] if columns else [],
        data=mongo_data, 
        page_size=10,
        sort_action='native',
        filter_action='native',
        page_action='native',
        fixed_rows={'headers': True},
        style_table={'height': '400px', 'overflowY': 'auto'},
        style_cell={'textAlign': 'left', 'minWidth': '150px', 'width': 'auto'},
        style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'}
    ),
    html.Br(),
    html.Div(id='map-id', style={'height': '500px'})
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('filtered-results', 'children'),
     Output('animal-type-graph', 'figure')],
    [Input('age-dropdown', 'value'),
     Input('breed-dropdown', 'value'),
     Input('type-dropdown', 'value')]
)
def filter_data(selected_age, selected_breed, selected_type):
    # Use the global list of data
    filtered_data = mongo_data
    
    if selected_age:
        filtered_data = [d for d in filtered_data if d.get('age_upon_outcome') == selected_age]
    if selected_breed:
        filtered_data = [d for d in filtered_data if d.get('breed') == selected_breed]
    if selected_type:
        filtered_data = [d for d in filtered_data if d.get('animal_type') == selected_type]

    type_counts = {}
    for item in filtered_data:
        atype = item.get('animal_type')
        if atype:
            type_counts[atype] = type_counts.get(atype, 0) + 1
            
    x_values = list(type_counts.keys())
    y_values = list(type_counts.values())

    figure = {
        'data': [
            {
                'x': x_values,
                'y': y_values,
                'type': 'bar',
                'name': 'Animal Type Counts'
            }
        ],
        'layout': {
            'title': 'Number of Each Animal Type',
            'xaxis': {'title': 'Animal Type'},
            'yaxis': {'title': 'Count'}
        }
    }
    
    return html.P(f"Filtered {len(filtered_data)} records."), figure

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "data"),
     Input('datatable-id', "active_cell")]
)
def update_map(data, active_cell):
    # 'data' here is the filtered/paginated data *currently* displayed in the table.
    if active_cell and 'row' in active_cell:
        row_index = active_cell['row']
        # Use the data that is currently visible in the table
        if row_index < len(data): 
            selected_row = data[row_index]
            lat = selected_row.get('location_lat')
            lon = selected_row.get('location_long')
            
            # Default location for mapping if lat/lon are missing
            default_lat = 30.2672 # Austin, TX
            default_lon = -97.7431
            
            if lat and lon:
                geo_tile_url = "https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png"
                return dl.Map(center=[lat, lon], zoom=12, children=[
                    dl.TileLayer(url=geo_tile_url, attribution="&copy; OpenTopoMap contributors"),
                    dl.Marker(position=[lat, lon], draggable=False)
                ], style={'height': '100%'})
            else:
                return dl.Map(center=[default_lat, default_lon], zoom=10, children=[
                    dl.TileLayer(url="https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png", attribution="&copy; OpenTopoMap contributors"),
                ], style={'height': '100%'}), html.P("Location data is not available for this entry. Map centered on Austin.")
                
    # Default map view when no row is selected
    default_lat = 30.2672 
    default_lon = -97.7431
    return dl.Map(center=[default_lat, default_lon], zoom=10, children=[
        dl.TileLayer(url="https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png", attribution="&copy; OpenTopoMap contributors"),
    ], style={'height': '100%'}), html.P("Click on an animal entry to display its location on the map.")


if __name__ == '__main__':
    print("Starting server at http://127.0.0.1:8050 ...")
    try:
        app.run(debug=False, port=8050)
    except Exception as e:
        print(f"\nCRITICAL EXECUTION ERROR: {e}")
        traceback.print_exc()

Importing libraries...
Libraries imported successfully.
Reading data from sample_animal_data.txt...
Loaded 8 records from sample_animal_data.txt.
Setting up Dash layout...
Starting server at http://127.0.0.1:8050 ...


In [ ]:
!python -m pip install numpy==1.24.4

  Using cached numpy-1.24.4.tar.gz (10.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "c:\Users\alexa\AppData\Local\Programs\Python\Python312-32\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
        File "c:\Users\alexa\AppData\Local\Programs\Python\Python312-32\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\alexa\AppData\Local\Programs\Python\Python312-32\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 137, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "c:\Users\alexa\AppData\Local\Programs\Python\Python3